In [1]:
import re
import numpy as np
import copy

text = "Let’s say the elements to be deleted is known, instead of the indexes of those elements."

################################################
# tokenize sentence and make a word list
################################################

#lookahead
temp = re.split("(?=[^a-zA-Z-'’])", text)
temp = [t.lstrip() for t in temp]

#lookback
split_text = []
for t in temp:
    split_text += re.split("(?<=[\"])",t)

# create a list that shows 1 if a sign is neighboring the same sign; otherwise 0
cnt = 0
neighbor_list = []
for i in range(len(split_text)):
    
    if i == 0:
        if split_text[i]==split_text[i+1] and re.match(r"^[^a-zA-Z].*[^a-zA-Z]*$", split_text[i]):
            neighbor_list.append(1)
        else:
            neighbor_list.append(0)
        
    if i != 0 and i != len(split_text)-1:  
        if split_text[i-1]==split_text[i] and split_text[i]==split_text[i+1] and re.match(r"^[^a-zA-Z].*[^a-zA-Z]*$", split_text[i]):
            neighbor_list.append(1)
        elif split_text[i-1]==split_text[i] and split_text[i]!=split_text[i+1] and re.match(r"^[^a-zA-Z].*[^a-zA-Z]*$", split_text[i]):
            neighbor_list.append(1)     
        elif split_text[i-1]!=split_text[i] and split_text[i]==split_text[i+1] and re.match(r"^[^a-zA-Z].*[^a-zA-Z]*$", split_text[i]):
            neighbor_list.append(1)
        else:
            neighbor_list.append(0)
            
    if i == len(split_text)-1:
        if split_text[i-1]==split_text[i]:
            neighbor_list.append(1)
        else:
            neighbor_list.append(0)        

neighbor_list = [1 if i > 0 else 0 for i in neighbor_list]

# create a list of indices of consecutive signs (2nd and later) and its count
index_list = []
cnt_list = []
cnt = 0

for i, z in enumerate(zip(neighbor_list, split_text)):
    n = z[0]
    s = z[1]

    if neighbor_list[i-1] == 1 and n == 1:
        cnt += 1
        if cnt > 0:
            index_list.append(i)
            cnt_list.append(cnt + 1)
    else:
        if cnt > 0:
            print()
        cnt = 0

for i, c in enumerate(cnt_list):
    if i == 0:
        cnt_list[0] = 0
        index_list[0] = 0
        
    elif i != len(cnt_list)-1:
        if cnt_list[i] > cnt_list[i+1]:
            pass
        else:
            cnt_list[i] = 0
            index_list[i] = 0

# create a list of indices of consecutive signs (last one only) and its count
cnt_list_final = [c for c in cnt_list if c != 0]
index_list_final = [i for i in index_list if i != 0]

# gather all consecutive signs to its last one
for i, n in enumerate(index_list_final):
    split_text[n] = split_text[n] * cnt_list_final[i]
    
# delete preceeding ones
for c, i in zip(cnt_list_final[::-1], index_list_final[::-1]):
    start = i-c+1
    end = i
    
    del split_text[start:end]
    


################################################
# 1 if word, 0 if non-word or punc
################################################

word_mask = []
for w in split_text:
    if re.match(r'[a-zA-Z]', w):
        word_mask.append(1)
    else:
        word_mask.append(0)

################################################
# 1 if word, 0 if non-word or punc
################################################

word_mask = []
for w in split_text:
    if re.match(r'[a-zA-Z]', w):
        word_mask.append(1)
    else:
        word_mask.append(0)
        
################################################
# tokenized word list with punc excluded
################################################
text_words = []
for i, w in enumerate(split_text):
    if word_mask[i] == 1:
        text_words.append(w)

letter_num_list = [] 
for n in text_words:
    letter_num_list.append(len(n))
    
################################################
#Argsort based on word length,
#and give the first word the least priority
################################################
    
# argsort to determine priority order of words to drop
letter_num_list = np.array(letter_num_list)
# change to descending order
letters_num_argsort = np.argsort(letter_num_list)[::-1]
# drop 0 from array and append to the last
letters_num_argsort = letters_num_argsort[letters_num_argsort!=0]
letters_num_argsort = np.append(letters_num_argsort, 0)

################################################
#If 3 or less words, drop all words
################################################

#initialize list
args_to_drop = []

if len(letters_num_argsort) <= 3:
    args_to_drop = list(letters_num_argsort)

###############################################
#Otherwise, based on letters_num_argsort, 
#select 3 words that are NOT next to each other
###############################################

else:

    #select the 1st word to hide
    args_to_drop.append(letters_num_argsort[0])

    #select the 2nd word to hide
    for i in range(1, len(letters_num_argsort)):
        if abs(args_to_drop[0] - letters_num_argsort[i]) != 1:
            args_to_drop.append(letters_num_argsort[i])
            #remember arg of the selected element
            arg_second = i
            break
                
    #select the 3rd word to hide
    for i in range(arg_second+1, len(letters_num_argsort)):
        if (abs(args_to_drop[0] - letters_num_argsort[i]) != 1) and (abs(args_to_drop[1] - letters_num_argsort[i]) != 1):
            args_to_drop.append(letters_num_argsort[i])
            break
        # if 3 cannot be selected, compromise and fill it with any
        else: 
            letters_num_argsort = list(letters_num_argsort)
            
            letters_num_argsort.remove(args_to_drop[0])
            letters_num_argsort.append(args_to_drop[0])
            letters_num_argsort.remove(args_to_drop[1])
            letters_num_argsort.append(args_to_drop[1])
            
            # see if a number that is not neighboring the first 2 can be selected
            for num in letters_num_argsort:
                if num != 0:                
                    if (num != args_to_drop[0]+1) and (num != args_to_drop[0]-1) and (num != args_to_drop[1]+1) and (num != args_to_drop[1]-1):
                        args_to_drop.append(num)
                        break
                else:
                    break
            
            # if still only 2, pick the one right before 0 in argsort
            try:
                if len(args_to_drop) == 2:
                    args_to_drop.append(letters_num_argsort[letters_num_argsort.index(0)-1])  
            except:
                # as the very last resort, pick the first word of the sentence
                args_to_drop.append(0)              
            break 
               
####################################################
#convert arg in text_words into arg in split_text
####################################################
            
args_to_drop_all = []
for idx in args_to_drop:
    cnt = 0
    for i in range(len(word_mask)):
        if cnt < idx:    
            if word_mask[i]==1:
                cnt+=1
        elif cnt == idx:
            if word_mask[i]==1:
                args_to_drop_all.append(i)
                break
                
####################################################
#reconstruct sentence with blanks
####################################################

split_text_before = copy.deepcopy(split_text)

for i in args_to_drop_all:
    split_text[i]= "___"
    
reconstructed_text = " ".join(split_text)

#remove space after the starting doble quotation
reconstructed_text = re.sub(r'" ', '"', reconstructed_text)

reconstructed_text = re.sub(r'([a-zA-Z].*[a-zA-Z]) ([":;,.!?]+) (.*)', r'\1\2 \3', reconstructed_text)
reconstructed_text = re.sub(r'([a-zA-Z].*[a-zA-Z]) ([":;,.!?]) (.*)', r'\1\2 \3', reconstructed_text)
reconstructed_text = re.sub(r'([a-zA-Z].*[a-zA-Z]) ([":;,.!?])$', r'\1\2', reconstructed_text)

print("Original:",text)
print()
print("Question:",reconstructed_text)
print()

#############################################################
##randomize cards so that they won't match the correct order
#############################################################

card_order = args_to_drop_all

while card_order == sorted(args_to_drop_all):
    if len(card_order) >= 3:
        card_order = random.sample(args_to_drop_all , 3)
    elif len(card_order) == 2:
        card_order = random.sample(args_to_drop_all , 2)
    elif len(card_order) == 1:
        card_order = random.sample(args_to_drop_all , 1)

if len(card_order) > 2:
    print(f'[{split_text_before[card_order[0]]}]', f'[{split_text_before[card_order[1]]}]', f'[{split_text_before[card_order[2]]}]')
elif len(card_order) == 2:
    print(f'[{split_text_before[card_order[0]]}]', f'[{split_text_before[card_order[1]]}]')
elif len(card_order) == 1:
    print(f'[{split_text_before[card_order[0]]}]')

Original: Let’s say the elements to be deleted is known, instead of the indexes of those elements.

Question: Let’s say the ___ to be deleted is known, instead of the ___ of those ___ .

[elements] [elements] [indexes]
